<a href="https://colab.research.google.com/github/celiahc823/Machine_Learning/blob/main/Predicting_Movie_rental_durations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [8]:
file_path = '/content/drive/MyDrive/Colab Notebooks/rental_info.csv'  # Define the file path
df = pd.read_csv(file_path)

In [9]:
# Convertir rental_date and return_date tipo datetime
df["return_date"] = pd.to_datetime(df["return_date"])
df["rental_date"] = pd.to_datetime(df["rental_date"])

In [20]:
#diferencia en días entre la fecha de devolución y la fecha de alquiler y crea una nueva columna llamada rental_length_days para almacenar esta diferencia
df["rental_length_days"] = (df["return_date"] - df["rental_date"]).dt.days

In [21]:
# Creación de Variables Dummy para special_features
df["deleted_scenes"] = df["special_features"].apply(lambda x: 1 if "Deleted Scenes" in str(x) else 0)
df["behind_the_scenes"] = df["special_features"].apply(lambda x: 1 if "Behind the Scenes" in str(x) else 0)

In [22]:
# excluyendo las columnas rental_date, return_date, special_features, y rental_length_days, ya que podrían generar fugas de datos
columns_to_exclude = ["rental_date", "return_date", "special_features", "rental_length_days"]
feature_columns = [col for col in df.columns if col not in columns_to_exclude]


In [13]:
X = df[feature_columns]
y = df["rental_length_days"]

In [14]:
# División del Conjunto de Datos en Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [16]:
# Modelo
models = {
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=9),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=9)
}

# Entrenamiento y Evaluación de Modelos
best_mse = float("inf")
best_model = None
model_performance = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    model_performance[name] = mse

    if mse < best_mse:
        best_mse = mse
        best_model = model



In [ ]:
# Entrenamiento y Evaluación de Modelos
best_mse = float("inf")
best_model = None
model_performance = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    model_performance[name] = mse

    if mse < best_mse:
        best_mse = mse
        best_model = model


In [23]:
# Mejor Modelo
output_dir = "./data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



In [24]:
# Save the best model if it meets the MSE criteria
if best_mse <= 3:
    joblib.dump(best_model, os.path.join(output_dir, "best_model.pkl"))

In [25]:
print(f"Best Model: {best_model}")
print(f"Best MSE: {best_mse:.3f}")

Best Model: RandomForestRegressor(random_state=9)
Best MSE: 2.030
